In [ ]:
# Modules to import

!pip install --user nipy

# DA AGGIUNGERE NEL NUOVO CODICE

In [1]:
import tensorflow as tf
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)
with tf.Session() as sess:
    print (sess.run(c))

[[22. 28.]
 [49. 64.]]


In [ ]:
!pip install --user keras

In [ ]:
!pip install --user nilearn

In [ ]:
!pip install --user seaborn

In [ ]:
# Main Imports

import nilearn

from nilearn.image import resample_to_img

import pylab as plt

import numpy as np
import nibabel as nb
import os
import glob
import random
import pandas as pd
import re

import seaborn as sns #added
sns.set(style="darkgrid") #added



In [ ]:
# Local directory where the Drive is mounted
rootDirectory = "/data/RMN/LUCA_PASQUINI"

# Local subdirectory where dataset is mounted
dataDir = "DATI_SEGMENTATI_SCALATI_media"

# Dataset dir
datasetDir = f"{rootDirectory}/{dataDir}"

datasetDir

In [ ]:
from nilearn.image import mean_img #added
from nilearn.plotting import plot_anat #added

In [ ]:
ls {datasetDir}/

# Survival Data

In order to associate survival data we read a CSV file provided with 0/1 for each subject

In [ ]:
rm {datasetDir}/Def_Labels.csv*

In [ ]:
rm {datasetDir}/Def_Labels_array.csv*

In [ ]:
ls {datasetDir}/

In [ ]:
#thing=['Labels_array.csv*']
#for name in glob.glob(f"{datasetDir}/*"):
    #print(name)
 #   folders = os.path.basename(name)
    #print(folders)
  #  if thing in folders:
#        folders.remove(thing)

In [ ]:
ls {datasetDir}/BIAVATI_S/ROI/SOLID.nii*

In [ ]:
fileName = "/data/RMN/LUCA_PASQUINI/DATI_SEGMENTATI_SCALATI_media/Array_Labels_Def.csv"

In [ ]:
df = pd.read_csv(fileName, sep=";", header=None)

In [ ]:
df.columns = ["Subject", "Survival"]

In [ ]:
df = df.set_index('Subject')

In [ ]:
# Dump Survival data
#print(df)

pd.set_option('display.max_rows', len(df))
print(df)

In [ ]:
#def normalize_name(subject_path, add_name=False):
    #Components= []
#    subject = os.path.basename(subject_path)
 #   subject = re.sub(r'^(DE|D|DI|LO|DEL)_', '', subject)
 #   components =  subject.split("_")
#    if len(components) > 1 and add_name:
 #       return components[0].title() + components[1][0]
 #   else:
 #       return components[0].title()

#def get_subject_metadata(subject_path, subjects_with_name=[]):
  #  dirname = os.path.basename(subject_path)
  #  return (dirname, normalize_name(subject_path, add_name=dirname in subjects_with_name))
 



In [ ]:
def normalize_name(subject_path, add_name):
    #Components= []
    subject = os.path.basename(subject_path)
    subject = re.sub(r'^(DE|D|DI|LO|DEL)_', '', subject)
    components =  subject.split("_")
    if add_name[0]==components[0] or add_name[1]==components[0] or add_name[2]==components[0] :
            return components[0].title() + components[1][0].capitalize()
    else:
            return components[0].title()
        

def get_subject_metadata(subject_path, subjects_with_name=[]):
    dirname = os.path.basename(subject_path)
    return (dirname, normalize_name(subject_path, add_name=[name for name in subjects_with_name]))
 



In [ ]:
#subject=["EMANUELA_T","LUCA_PASQUINI","LUCA_L","MICHELE_M"]
#Components= []
#for i in range(len(subject)):
#    components =  subject[i].split("_")
#    Components.append(components)
#    
#print(Components)
#r=len(Components)
#for i in range(r):
#    for j in range(r):
#        if j!=i:
#            if Components[i][0]==Components[j][0]:
#                name=Components[i][0].title() + Components[i][1][0].capitalize()
#            else:
#                name=Components[i][0].title()
#                
                
        

In [ ]:
#functions application
Subject_dirs=[]
for subject_path in glob.glob(f"{datasetDir}/*"):
    subjects_dirs=[get_subject_metadata(subject_path , subjects_with_name=["BIANCHI","BOVE","PROIETTI"])]
    #print(subjects_dirs)
    

In [ ]:
Df_Subjects_dirs = pd.DataFrame()
for subject_path in glob.glob(f"{datasetDir}/*"):
    subjects_dirs=[get_subject_metadata(subject_path , subjects_with_name=["BIANCHI","BOVE","PROIETTI"])]
    print(subjects_dirs)
    df_subject_dirs = pd.DataFrame(subjects_dirs,columns=["Path", "Subject"])
    Df_Subjects_dirs=pd.concat([Df_Subjects_dirs,df_subject_dirs],ignore_index=True)

pd.set_option('display.max_rows', len(Df_Subjects_dirs))
print(Df_Subjects_dirs)


Subjects name must be normalized to encounter directory names

In [ ]:
#Df_Subjects_dirs = pd.DataFrame()
#for i in range(len(Subjects_dirs)):
#    df_subject_dirs = pd.DataFrame(Subjects_dirs[i],columns=["Path", "Subject"])
#    Df_Subjects_dirs=pd.concat([Df_Subjects_dirs,df_subject_dirs],ignore_index=True)
        
        
              
#pd.set_option('display.max_rows', len(Df_Subjects_dirs))
#print(Df_Subjects_dirs)

In [ ]:
Df_Subjects_dirs = Df_Subjects_dirs.set_index('Subject')
pd.set_option('display.max_rows', len(Df_Subjects_dirs))

print(Df_Subjects_dirs)

In [ ]:
x=Df_Subjects_dirs
x.drop(index='Array', columns='Path')
# Delete rows with index label a & b    
modX= x.drop(['Pascal','Mitchell','Rufini','Farella','Array'])
#modX["Path"]
pd.set_option('display.max_rows', len(modX))
modX = pd.DataFrame(modX)
print(modX)

In [ ]:
result = modX.join(df, on='Subject')

pd.set_option('display.max_rows', len(result))
result
    

In [ ]:
NAN_index=result['Survival'].index[result['Survival'].apply(np.isnan)]

df_index=result.index.values.tolist()
int_index=[df_index.index(i) for i in NAN_index]
int_index

In [ ]:
label=[df.iloc[x]['Survival'] for x in int_index]
label

In [ ]:
for l in label:
    result['Survival'].fillna(l,inplace=True)
pd.set_option('display.max_rows', len(result))
result

In [ ]:
'''' Get index positions of value in dataframe i.e. dfObj.'''
def getIndexes(dfObj, value):
    listOfPos = list()
    result = dfObj.isin([value]) # Get bool dataframe with True at positions where the given value exists
    seriesObj = result.any() # Get list of columns that contains the value
    columnNames = list(seriesObj[seriesObj == True].index)
    for col in columnNames: # Iterate over list of columns and fetch the rows indexes where value exists
        rows = list(result[col][result[col] == True].index)
        for row in rows:
            listOfPos.append((row, col))
# Return a list of tuples indicating the positions of value in the dataframe
    return listOfPos

In [ ]:
#print(modX)
#print(df)

#modX.shape
#df.shape

In [ ]:
listOfElems=[]
Dim = []
Data = []
IMG=[]
for Path in result["Path"]:
    #print(Path)
    if os.path.isfile(f"{datasetDir}/{Path}/T1_registered.nii.gz"):

        IMG_reg = nb.load(f"{datasetDir}/{Path}/T1_registered.nii.gz")
        DATA= IMG_reg.get_data()
        a = [DATA.shape]
        Dim.append(a)
        Data.append(DATA)
        IMG.append(IMG_reg)
        

        IMG_roi = nb.load(f"{datasetDir}/{Path}/ROI/SOLID.nii")
        ROI_DATA=IMG_roi.get_data()
        b=[ROI_DATA.shape]
        Dim.append(b)
        Data.append(ROI_DATA)
        IMG.append(IMG_roi)
        
        path=[f"{Path}"]
        listOfElems.append(path)

In [ ]:
len(listOfElems)

In [ ]:
listOfElems=[]
Dim = []
Data = []
IMG=[]
for Path in result["Path"]:
    #print(Path)
    if os.path.isfile(f"{datasetDir}/{Path}/T1_registered.nii.gz"):

        IMG_reg = nb.load(f"{datasetDir}/{Path}/T1_registered.nii.gz")
        DATA= IMG_reg.get_data()
        a = [DATA.shape]
        Dim.append(a)
        Data.append(DATA)
        IMG.append(IMG_reg)
        

        IMG_roi = nb.load(f"{datasetDir}/{Path}/ROI/SOLID.nii")
        ROI_DATA=IMG_roi.get_data()
        b=[ROI_DATA.shape]
        Dim.append(b)
        Data.append(ROI_DATA)
        IMG.append(IMG_roi)
        
        path=[f"{Path}"]
        listOfElems.append(path)
        
    else:
        if os.path.isfile(f"{datasetDir}/{Path}/T1.nii"):
            IMG_reg = nb.load(f"{datasetDir}/{Path}/T1.nii")
            DATA= IMG_reg.get_data()
            a = [DATA.shape]
            Dim.append(a)
            Data.append(DATA)
            IMG.append(IMG_reg)
            
            IMG_roi = nb.load(f"{datasetDir}/{Path}/ROI/SOLID.nii")
            ROI_DATA=IMG_roi.get_data()
            b=[ROI_DATA.shape]
            Dim.append(b)
            Data.append(ROI_DATA)
            IMG.append(IMG_roi)
            
            path=[f"{Path}"]
            listOfElems.append(path)

In [ ]:
len(listOfElems)
print(listOfElems)

In [ ]:
len(listOfElems)
listOfElems[6][0]

In [ ]:
#df.loc["Santis"]


In [ ]:
T1_Subject=[]
for i in range(len(listOfElems)):
    Pos=getIndexes(result, listOfElems[i][0])
    print(Pos[0][0])
    T1_Subject.append(Pos[0][0])
    
T1_Subject

In [ ]:
T1_Subject_array=np.asarray(T1_Subject)
#T1_Labels=[]
for i in range(len(T1_Subject_array)):
    try:
        # Subject found
        lab = df.loc[T1_Subject_array[i],"Survival"]
        print(f"Found {lab} for {T1_Subject_array[i]}")
    except:
        # Subject not found
        print(f"Subject {T1_Subject_array[i]} not fount")

In [ ]:
T1_Subject_array=np.asarray(T1_Subject)
T1_Labels=[]
for i in range(len(T1_Subject_array)):
    #try:
        # Subject found
        lab = result.loc[T1_Subject_array[i],"Survival"]
        T1_Labels.append(int(lab))
        #print(f"Found {lab} for {T1_Subject_array[i]}")
    #except:
        # Subject not found
        #print(f"Subject {T1_Subject_array[i]} not fount")
        
#T1_Labels=np.asarray(T1_Labels)
#T1_Labels


In [ ]:
Label_Def=[]
for x in range(len(T1_Labels)):
    label_Def=[[T1_Labels[x]]*2]
    Label_Def.append(label_Def)
    
Label_Def=np.asarray(Label_Def)
Label_Def=np.ravel(Label_Def)
Label_Def
#label_array=np.array([Label_Def[1][0][0],Label_Def[1][0][1]])
#label_array.shape
#for i in range(len(Label_Def)):
#    for j in range(2):
#        array_label=np.array([Label_Def[i][0][j]])
#print(array_label)

In [ ]:
#listOfElems=['MARAGNO_CLARA','MAIOLINI_SANTA']
#dictOfPos = {elem: getIndexes(result, elem[0]) for elem in listOfElems}
#print(dictOfPos)

In [ ]:
Dim=np.asarray(Dim)   
Data=np.asarray(Data)
IMG=np.asarray(IMG)

IMG[:][:].shape
print(Dim[219])
Data[219].shape
IMG[219].shape

In [ ]:
# results
Data.shape
IMG.shape
#L.shape

In [ ]:
#cerco i valori minimi sulla base dei quali fare il resample
Min_value=np.amin(Dim, axis=1)
Min_value
Min=np.amin(Min_value, axis=0)
Min
Min_value.shape
Min[2]
# Min_value[:,0]
# pos=np.where(Min_value[:,0]==Min[0])
# pos

In [ ]:

# [p.append(q.index(v)) if v in q else p.append(99999) for v in vm]


not_in_index = [x for x in range(Min_value.shape[0]) if not np.all(Min_value[x] == (192, 256, 144))]

not_in_index


In [ ]:
pos_1=np.where(Min_value[:,0]==192)
pos_1

pos_2=np.where(Min_value[:,1]==256)
pos_2

pos_3=np.where(Min_value[:,2]==144)
pos_3

eq=np.intersect1d(pos_1,pos_2)
eq

index_IMG=np.intersect1d(eq,pos_3)
index_IMG
def_index=random.choice(index_IMG)
print(def_index)

index_IMG

In [ ]:

for i in not_in_index:
    Res=nilearn.image.resample_to_img(IMG[i], IMG[def_index])
    IMG[i]=Res
    Data[i]=IMG[i].get_data()

        
#     Resample_results.append(Res_IMG)

# Prova_Data=np.ndarray.tolist(Data)

# for i in index_IMG:
#     Resample_results.append(Prova_Data[i])

# Resample_results=np.asarray(Resample_results)
# Resample_results.shape

In [ ]:
len(Data)

In [ ]:
Input_matrix=np.empty((len(Data),192,256,144)) 

for i in not_in_index:

    Input_matrix[i,:,:,:]=np.array(Data[i])




for i in index_IMG:

    Input_matrix[i,:,:,:]=np.array(Data[i])
    
    
    
Input_matrix.shape

# Input_matrix[20,47,10,20]

In [ ]:
#added block
## Create list of indices and shuffle them
N = Input_matrix.shape[0]
indices = np.arange(N)
np.random.shuffle(indices)

In [ ]:
#labels block added
labels=Label_Def
labels.shape

In [ ]:
#added block
#  Cut the dataset at 80% to create the training and test set
N_80p = int(0.8 * N)
indices_train = indices[:N_80p]
indices_test = indices[N_80p:]

In [ ]:
# Split the data into training and test sets
X_train = Input_matrix[indices_train, ...]
X_test = Input_matrix[indices_test, ...]

print(X_train.shape, X_test.shape)

In [ ]:
#outcome variable block added
y_train = labels[indices_train]
y_test = labels[indices_test]

In [ ]:
#block added to create the model
import tensorflow as tf
from keras.models import Sequential

from keras.layers import Dense, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

from keras.optimizers import Adam, SGD

from keras import backend as K

In [ ]:
#added block
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

y_train

In [ ]:
#block added to create the model
import tensorflow as tf
from keras.models import Sequential

from keras.layers import Dense, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

from keras.optimizers import Adam, SGD

from keras import backend as K

In [ ]:
# Get shape of input data
data_shape = tuple(X_train.shape[1:])

# Specify shape of convolution kernel
kernel_size = (3, 3)

# Specify number of output categories
n_classes = 2

# Specify number of filters per layer
filters = 16

data_shape

In [ ]:
#model block added
K.clear_session()
model = Sequential()

model.add(Conv2D(filters, kernel_size, activation='relu', input_shape=data_shape))
model.add(BatchNormalization())
model.add(MaxPooling2D())

model.add(Conv2D(filters * 2, kernel_size, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D())

model.add(Conv2D(filters * 4, kernel_size, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(n_classes, activation='softmax'))

# optimizer
learning_rate = 1e-5
adam = Adam(lr=learning_rate)
sgd = SGD(lr=learning_rate)

model.compile(loss='categorical_crossentropy',
              optimizer=adam, # swap out for sgd 
              metrics=['accuracy'])

model.summary()

In [ ]:
#added block
nEpochs = 100  # Increase this value for better results (i.e., more training)

In [ ]:
batch_size = 16   # Increasing this value might speed up fitting

In [ ]:
%time fit = model.fit(X_train, y_train, epochs=nEpochs, batch_size=batch_size)

In [ ]:
fig = plt.figure(figsize=(10, 4))
epoch = np.arange(nEpochs) + 1
fontsize = 16
plt.plot(epoch, fit.history['accuracy'], marker="o", linewidth=2,
         color="steelblue", label="accuracy")
plt.plot(epoch, fit.history['loss'], marker="o", linewidth=2,
         color="orange", label="loss")
plt.xlabel('epoch', fontsize=fontsize)
plt.xticks(fontsize=fontsize)
plt.yticks(fontsize=fontsize)
plt.legend(frameon=False, fontsize=16);